## ISID：説明性・解釈性のデモ（表形式データ）

## 1. 用意

①Azure Portalにて、Data Science Virtual Machine for Linux (Ubuntu)、でVMを作成

サイズはStandard NC6

②SSH接続

ターミナルで、

source activate py36

※仮想環境py36を起動


jupyter notebook password

※jupyter notebookのパスワードを設定


③ssh転送でポート8888をローカルマシンとつなぐ

jupyter notebook

で起動。

表示されるURLにアクセスします。 http://localhost:8888/tree


ローカルマシンのブラウザからVMのJupyterNotebookで作業


## 2. 実行環境の設定




In [ ]:
# 実行上問題ないwarningは非表示にする
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 乱数シードの固定
seed_value= 1234  # Seedの適当な値

# 1. pythonのシード固定
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
 
# 2. randomのシード固定
import random
random.seed(seed_value)
 
# 3. Numpyのシード固定
import numpy as np
np.random.seed(seed_value)

In [ ]:
# パッケージのimport
import pandas as pd
import sklearn

## 3. 各ライブラリのインストール

In [ ]:
# 最初はこちらを実行してパッケージをインストール
!pip install lime
!pip install shap
!pip install anchor_exp

In [ ]:
import lime, shap, anchor

In [ ]:
#print(lime.__version__)
print(shap.__version__)
#print(anchor.__version__)

In [ ]:
!pip show lime

In [ ]:
!pip show anchor_exp

## 4. タイタニックデータを用意

In [ ]:
# タイタニックデータ取得
# 参考
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/explain-model/tabular-data/advanced-feature-transformations-explain-local.ipynb

titanic_url = ('https://raw.githubusercontent.com/amueller/'
               'scipy-2017-sklearn/091d371/notebooks/datasets/titanic3.csv')
data = pd.read_csv(titanic_url)


In [ ]:
# データを作成
target_feature = ['survived']
numeric_features = ['age', 'fare','sibsp','parch']
categorical_features = ['embarked', 'sex', 'pclass']

df = data[target_feature+categorical_features + numeric_features]
y = data[target_feature].values
X = data[categorical_features + numeric_features]


In [ ]:
# データを確認
print(df.shape)
df.head()

# ※ageの1以下は乳幼児と思われる。11カ月 = 0.9167

In [ ]:
1/12*11

In [ ]:
y

## 5. データの前処理（型修正・欠損値処理）

In [ ]:
# データの型を確認
X.dtypes

In [ ]:
# pclassの型を修正
X["pclass"] = X["pclass"].astype(str)
X.dtypes


In [ ]:
# 欠損値のある列を確認
X.isnull().any(axis=0)

In [ ]:
# embarkedの欠損値を修正
X['embarked'] = X['embarked'].fillna("missing")
X.isnull().any(axis=0)

In [ ]:
# ageとfareの欠損値を修正
# データを分割
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 訓練データで中央値を求める
age_median = x_train["age"].median()
fare_median = x_train["fare"].median()

X["age"] = X["age"].fillna(age_median)
X["fare"] = X["fare"].fillna(fare_median)


In [ ]:
# 欠損値が全部消えたかを再確認
X.isnull().any(axis=0)

In [ ]:
# Xを確認する
X = pd.DataFrame(X)

# Anchorsで使うとき用に
X_original = X.copy()

X.head()



In [ ]:
#カテゴリデータを数値に変換する
from sklearn.preprocessing import LabelEncoder

feature_names = X.columns
categorical_features = [0,1,2]
categorical_names = {}

for feature in categorical_features:
    le = LabelEncoder()
    le.fit(X[feature_names[feature]])
    X.loc[:, feature_names[feature]] = le.transform(X[feature_names[feature]])
    categorical_names[feature] = le.classes_

X = X.astype(float)

In [ ]:
# カテゴリカルデータの変換を確認
categorical_names

In [ ]:
# Xを確認する
X = pd.DataFrame(X)
X.head()

In [ ]:
# 目的変数yの形を修正
y = y.reshape(-1)
print(y[0:5])

## 6. 機械学習で学習する

In [ ]:
# one-hot encodingを実施する
encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_features)
encoder.fit(X)


In [ ]:
# データ分割 ※前処理を同じ条件で行うこと
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
# 今回はランダムフォレストで学習としよう

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10, random_state=0)

# one-hot
encoded_X_train = encoder.transform(X_train)

# 学習
forest = forest.fit(encoded_X_train, y_train)


In [ ]:
# 検証データの性能

from sklearn.metrics import accuracy_score

encoded_X_val = encoder.transform(X_val)


print(accuracy_score(y_train, forest.predict(encoded_X_train).round()))
print(accuracy_score(y_val, forest.predict(encoded_X_val)).round())

print(X_val.shape)

In [ ]:
# 生存確率
pb = forest.predict_proba(encoded_X_val)


In [ ]:
# index=0の人の生存確率
pb[0]

In [ ]:
# index=1の人の生存確率
pb[1]

In [ ]:
# 中身を確認する
categorical_names

In [ ]:
# 中身の確認

X_val_show=pd.DataFrame((encoder.transform(X_val)).todense())

feature_names=['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_Missing','Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age','Fare','SibSp', 'Parch']

X_val_show.columns =feature_names
X_val_show.head()

# 7. LIME

In [ ]:
#LIMEの準備
import lime.lime_tabular
import re,itertools,json

In [ ]:
# 日本語表示用の設定

def visualize_instance_html_japanese(self, exp, label, div_name, exp_object_name,
                                text=True, opacity=True):
    if not text:
        return u''
    text = (self.indexed_string.raw_string()
            .encode('utf-8', 'xmlcharrefreplace').decode())
    text = re.sub(r'[<>&]', '|', text)
    exp = [(self.indexed_string.word(x[0]),
            self.indexed_string.string_position(x[0]),
            x[1]) for x in exp]
    all_ocurrences = list(itertools.chain.from_iterable(
        [itertools.product([x[0]], x[1], [x[2]]) for x in exp]))
    all_ocurrences = [(x[0], int(x[1]), x[2]) for x in all_ocurrences]
    ret = '''
        %s.show_raw_text(%s, %d, %s, %s, %s);
        ''' % (exp_object_name, json.dumps(all_ocurrences), label,
               json.dumps(text), div_name, json.dumps(opacity))
    return ret

In [ ]:
# LIME用の設定
feature_names = ['Embarked', 'Sex', 'Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
categorical_names

In [ ]:
categorical_features

In [ ]:
# 推論関数の定義
predict_fn = lambda x: forest.predict_proba(encoder.transform(x)).astype(float)


In [ ]:
# LIMEの用意

from lime.lime_text import TextDomainMapper
TextDomainMapper.visualize_instance_html = visualize_instance_html_japanese



explainer = lime.lime_tabular.LimeTabularExplainer(np.array(X_train) ,feature_names = feature_names,
                                                   class_names=["Not Survived","Survived"],
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3, verbose=True)



In [ ]:
# 局所説明性が欲しいデータをピックアップ
i = 0
X_val.iloc[i,:]

In [ ]:
# LIMEで局所説明
exp = explainer.explain_instance(np.array(X_val)[i], predict_fn, num_samples=1000)

# num_samplesが作成する模擬データ数

In [ ]:
# 表示
print(y_val[i])
exp.show_in_notebook(show_all=False)

In [ ]:
# 別のデータ

# 局所説明性が欲しいデータをピックアップ
i = 1

# LIMEで局所説明
exp = explainer.explain_instance(np.array(X_val)[i], predict_fn, num_samples=1000)

# 表示
print(y_val[i])
exp.show_in_notebook(show_all=False)

# 8. SHAP

In [ ]:
import shap

In [ ]:
# 推論関数の定義  ※LIMEと違って、one-hotにしてから扱う
predict_fn = lambda x: forest.predict_proba(x).astype(float)


In [ ]:
feature_names=['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_Missing','Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age','Fare','SibSp', 'Parch']

In [ ]:
# sparceな行列をdenseに
encoded_X_train = encoded_X_train.todense()
encoded_X_val = encoded_X_val.todense()

In [ ]:
# pandasに
encoded_X_train = pd.DataFrame(encoded_X_train)
encoded_X_val = pd.DataFrame(encoded_X_val)


encoded_X_train.columns =feature_names
encoded_X_val.columns =feature_names
encoded_X_val.head()

In [ ]:
ex = shap.KernelExplainer(predict_fn, encoded_X_train)

In [ ]:
# 全データだと多いのでkmeansで代表点を求めて使用する
# 参考　https://github.com/slundberg/shap/blob/master/notebooks/kernel_explainer/Diabetes%20regression.ipynb

# X_train_summary = shap.kmeans(encoded_X_train, 10)
# ex = shap.KernelExplainer(predict_fn, X_train_summary)

In [ ]:
# SHAPで局所説明の値を求める
i = 0
shap_values = ex.shap_values(encoded_X_val.iloc[i,:])


In [ ]:
print(shap_values[1])  # [1] は目的変数yが1 = survivedであるSHAPを求めている

In [ ]:
# 可視化  ※[1] は目的変数yが1 = survivedであるSHAPを求めている
shap.initjs()
shap.force_plot(ex.expected_value[1], shap_values[1], encoded_X_val.iloc[i,:])


In [ ]:
# 別のデータで
# 局所説明性が欲しいデータをピックアップ
i = 1

# SHAPで局所説明の値を求める
shap_values = ex.shap_values(encoded_X_val.iloc[i,:])

# shap_values
print(shap_values[1])

# 可視化  ※[1] は目的変数yが1 = survivedであるSHAPを求めている
shap.initjs()
shap.force_plot(ex.expected_value[1], shap_values[1], encoded_X_val.iloc[i,:])

In [ ]:
# one-hotにした影響力はどう計算するのか？
# 参考　Questions about SHAP handling categorical variables #397
# https://github.com/slundberg/shap/issues/397 

# You should use SUM (assuming you don't want to break it out by category). 
# Because that will measure the total effect of all the categories,
# and so capture the impact of that feature before one-hot encoding.


In [ ]:
# 計算する

#feature_names=['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_Missing','Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',
#       'Age','Fare','SibSp', 'Parch']

shap_Embarked = shap_values[1][0:4].sum()
shap_Sex = shap_values[1][4:6].sum()
shap_Pclass = shap_values[1][6:9].sum()
shap_Age = shap_values[1][9]
shap_Fare = shap_values[1][10]
shap_Sibsp = shap_values[1][11]
shap_Parch = shap_values[1][12]


In [ ]:
print(shap_Embarked)
print(shap_Sex)
print(shap_Pclass)
print(shap_Age)
print(shap_Fare)
print(shap_Sibsp)
print(shap_Parch)


# 9. Anchors

Anchorsでは連続値は扱えず、ビンに区切る必要がある

In [ ]:
from anchor import utils
from anchor import anchor_tabular

In [ ]:
# 上の方にある、Anchorsで使うとき用にからとってくる
X = X_original 
X.head()

In [ ]:
# 連続値を20分割
X['age_cut'] = pd.cut(df['age'], 20)
X['sibsp_cut'] = pd.cut(df['sibsp'], 20)
X['parch_cut'] = pd.cut(df['parch'], 20)
X['fare_cut'] = pd.cut(df['fare'], 20)

In [ ]:
X.head()

In [ ]:
# 列の順番入れ替え
X=X.ix[:,['embarked', 'sex', 'pclass', 'age_cut', 'fare_cut','sibsp_cut','parch_cut']]

In [ ]:
X.head()

In [ ]:
# 分割したものをテキストとして扱う
X['age_cut'] = X['age_cut'].astype(str)
X['sibsp_cut'] = X['sibsp_cut'].astype(str)
X['parch_cut'] = X['parch_cut'].astype(str)
X['fare_cut'] = X['fare_cut'].astype(str)


In [ ]:
#カテゴリデータを数値に変換する
feature_names = X.columns
categorical_features = [0,1,2,3,4,5,6]
categorical_names = {}

for feature in categorical_features:
    le = LabelEncoder()
    le.fit(X[feature_names[feature]])
    X.loc[:, feature_names[feature]] = le.transform(X[feature_names[feature]])
    categorical_names[feature] = le.classes_

X = X.astype(int)

In [ ]:
X.head()

In [ ]:
# カテゴリカルデータの対応を確認
categorical_names

In [ ]:
feature_names

In [ ]:
class_names=["0_Survived", "1_Survived"]

In [ ]:
# 機械学習を実施

In [ ]:
# データ分割 ※前処理を同じ条件で行うこと
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
# 
# https://github.com/marcotcr/anchor/blob/master/anchor/anchor_tabular.py
explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, np.array(X_train), categorical_names)
explainer.fit(np.array(X_train), y_train, np.array(X_val), y_val)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10, random_state=0)

forest.fit(explainer.encoder.transform(np.array(X_train)),y_train)

# 本当はone-hotにすべきで連続ではないが、今回は面倒なのでこのままいく

In [ ]:
# 推論関数の定義
predict_fn = lambda x: forest.predict(explainer.encoder.transform(x))

In [ ]:
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(np.array(X_train))))
print('Test', sklearn.metrics.accuracy_score(y_val, predict_fn(np.array(X_val))))

In [ ]:
# 局所説明を得る

i = 0
forest.predict_proba(explainer.encoder.transform(np.array(X_val)))[i]
print('Prediction: ', explainer.class_names[predict_fn(np.array(X_val)[i].reshape(1, -1))[0]])  

In [ ]:
exp = explainer.explain_instance(np.array(X_val)[i].astype(int), forest.predict, threshold=0.95)

In [ ]:
exp.show_in_notebook()

In [ ]:
# 別のデータの場合、局所説明を得る

i = 1
forest.predict_proba(explainer.encoder.transform(np.array(X_val)))[i]
print('Prediction: ', explainer.class_names[predict_fn(np.array(X_val)[i].reshape(1, -1))[0]])
exp = explainer.explain_instance(np.array(X_val)[i].astype(int), forest.predict, threshold=0.95)
exp.show_in_notebook()